# FastAI



In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

### Utils


In [ ]:
import zipfile

### Mount drive


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
# os.mkdir('/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Kaggle')

### Use Kaggle API


In [ ]:
pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd gdrive/Shareddrives

/content/gdrive/Shareddrives


In [ ]:
%cd 'TC3007C Inteligencia Artificial/Kaggle'

[Errno 2] No such file or directory: 'TC3007C Inteligencia Artificial/Kaggle'
/content/gdrive/Shareddrives


In [ ]:
import pathlib
file = pathlib.Path("hotel-id-to-combat-human-trafficking-2022-fgvc9.zip")
if file.exists():
    print("File exist")
else:
    print("File not exist")
    !kaggle competitions download -c hotel-id-to-combat-human-trafficking-2022-fgvc9
    if file.exists():
        # Unzip
        for file in os.listdir():
          if file.endswith(".zip"):
              with zipfile.ZipFile(file, "r") as zip_file:
                  zip_file.extractall()
              os.remove(file)


File not exist
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /content/gdrive/MyDrive/Kaggle. Or use the environment method.


### Install and import FastAI


In [ ]:
!pip install -Uq fastai2
from fastai.vision.all import *

     |████████████████████████████████| 179 kB 6.9 MB/s 


In [ ]:
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = "/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Data/data_3_pipe"
fns = get_image_files(path)

In [ ]:
class DataLoaders(GetAttr):
  def __init__(self, *loaders): self.loaders = loaders
  
  def __getitem__(self, i): return self.loaders[i]

  train,valid = add_props(lambda i,self: self[i])

In [ ]:
hotels = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files,
  splitter=RandomSplitter(valid_pct=0.2, seed=42),
  get_y=parent_label,
  item_tfms=Resize(128))

In [ ]:
dls = hotels.dataloaders(path)

### Optimizer Documentation

In [ ]:
#link Optimizers: https://docs.fast.ai/optimizer.html
#link Vision Learner: https://docs.fast.ai/vision.learner.html#vision_learner
#link AdamW: 

In [ ]:
add_docs(Optimizer, 
         zero_grad="Standard PyTorch API: Zero all the grad attributes of the parameters",
         step="Standard PyTorch API: Update the stats and execute the steppers in on all parameters that have a grad",
         state_dict="Return the state of the optimizer in a dictionary",
         load_state_dict="Load the content of `sd`",
         clear_state="Reset the state of the optimizer")

In [ ]:
opt = Optimizer([1,2,3], noop)
test_eq(opt.param_lists, [[1,2,3]])
opt = Optimizer(range(3), noop)
test_eq(opt.param_lists, [[0,1,2]])
opt = Optimizer([[1,2],[3]], noop)
test_eq(opt.param_lists, [[1,2],[3]])
opt = Optimizer(([o,o+1] for o in range(0,4,2)), noop)
test_eq(opt.param_lists, [[0,1],[2,3]])

In [ ]:
def tst_arg(p, lr=0, **kwargs): return p
tst_arg.defaults = dict(lr=1e-2)

def tst_arg2(p, lr2=0, **kwargs): return p
tst_arg2.defaults = dict(lr2=1e-3)

def tst_arg3(p, mom=0, **kwargs): return p
tst_arg3.defaults = dict(mom=0.9)

def tst_arg4(p, **kwargs): return p

opt = Optimizer([1,2,3], [tst_arg,tst_arg2, tst_arg3])
test_eq(opt.hypers, [{'lr2': 1e-3, 'mom': 0.9, 'lr': 1e-2}])
opt = Optimizer([1,2,3], tst_arg, lr=0.1)
test_eq(opt.hypers, [{'lr': 0.1}])
opt = Optimizer([[1,2],[3]], tst_arg)
test_eq(opt.hypers, [{'lr': 1e-2}, {'lr': 1e-2}])
opt = Optimizer([[1,2],[3]], tst_arg, lr=0.1)
test_eq(opt.hypers, [{'lr': 0.1}, {'lr': 0.1}])

In [ ]:
opt = Optimizer([[1,2],[3]], tst_arg, lr=[0.1,0.2])
test_eq(opt.hypers, [{'lr': 0.1}, {'lr': 0.2}])
opt = Optimizer([[1,2],[3],[4]], tst_arg, lr=slice(1e-2))
test_eq(opt.hypers, [{'lr': 1e-3}, {'lr': 1e-3}, {'lr': 1e-2}])
opt = Optimizer([[1,2],[3],[4]], tst_arg, lr=slice(1e-4,1e-2))
test_eq(opt.hypers, [{'lr': 1e-4}, {'lr': 1e-3}, {'lr': 1e-2}])
test_eq(opt.param_groups, [{'params': [1,2], 'lr': 1e-4}, {'params': [3], 'lr': 1e-3}, {'params': [4], 'lr': 1e-2}])
test_fail(lambda: Optimizer([[1,2],[3],[4]], tst_arg, lr=np.array([0.1,0.2])))

In [ ]:
def tst_param(val, grad=None):
    "Create a tensor with `val` and a gradient of `grad` for testing"
    res = tensor([val]).float()
    res.grad = tensor([val/10 if grad is None else grad]).float()
    return res

In [ ]:
p = tst_param(1., 0.1)
sgd_step(p, 1.)
test_eq(p, tensor([0.9]))
test_eq(p.grad, tensor([0.1]))

In [ ]:
params = tst_param([1,2,3], [0.1,0.2,0.3])
opt = Adam(pa, lr=0.1, wd=0)
opt.step()
step = -0.1 * 0.1 / (math.sqrt(0.1**2) + 1e-8)
test_close(params[0], tensor([1+step, 2+step, 3+step]))
opt.step()
test_close(params[0], tensor([1+2*step, 2+2*step, 3+2*step]), eps=1e-3)

NameError: ignored

### QHAdam

In [ ]:
params = tst_param([1,2,3], [0.1,0.2,0.3])
opt = QHAdam(params, lr=0.1)
opt.step()
step = -0.1 * (((1-0.7) * 0.1) + (0.7 * 0.1)) / (
     math.sqrt(((1-1.0) * 0.1**2) + (1.0 * 0.1**2)) + 1e-8) 
test_close(params[0], tensor([1+step, 2+step, 3+step]))
opt.step()
test_close(params[0], tensor([1+2*step, 2+2*step, 3+2*step]), eps=1e-3)

In [ ]:
import torch, torchvision


learn = vision_learner(dls, resnet18, QHAdam(params, lr = 0.001), metrics=[accuracy,error_rate], pretrained=True)
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,2.020010,1.356955,0.533333,0.466667,00:09


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.407934,1.115113,0.600000,0.400000,00:13
1,1.180034,0.849017,0.600000,0.400000,00:13
2,1.042891,0.656189,0.683333,0.316667,00:13
3,0.881524,0.527953,0.783333,0.216667,00:13
4,0.754279,0.444106,0.800000,0.200000,00:13
5,0.638586,0.386672,0.850000,0.150000,00:13
6,0.545350,0.354233,0.883333,0.116667,00:13
7,0.469517,0.321755,0.883333,0.116667,00:13
8,0.408942,0.318832,0.883333,0.116667,00:13
9,0.357573,0.318399,0.883333,0.116667,00:14


### AdamW

In [ ]:
import tensorflow as tf

In [ ]:
adamW = tf.keras.optimizers.experimental.AdamW(
    learning_rate = 0.001,
    weight_decay = 0.004,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon= 1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='AdamW'
)

In [ ]:
  learn = cnn_learner(dls, resnet18, adamW, metrics=[accuracy,error_rate], pretrained=True)
learn.fine_tune(60)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,2.006193,1.825195,0.333333,0.666667,00:09


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.152454,1.383919,0.483333,0.516667,00:13
1,1.028548,1.021589,0.583333,0.416667,00:13
2,0.913559,0.799972,0.650000,0.350000,00:13
3,0.813889,0.624454,0.733333,0.266667,00:13
4,0.720886,0.508981,0.800000,0.200000,00:14
5,0.620709,0.432434,0.816667,0.183333,00:13
6,0.537087,0.375744,0.866667,0.133333,00:13
7,0.467460,0.335160,0.883333,0.116667,00:13
8,0.408942,0.308196,0.883333,0.116667,00:13
9,0.361394,0.291627,0.883333,0.116667,00:13


### Adam


In [ ]:
import torch, torchvision

learn = cnn_learner(dls, resnet18, Adam(lr = 0.001), metrics=[accuracy,error_rate], pretrained=True)
learn.fine_tune(30)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Res

  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.961864,2.633617,0.366667,0.633333,00:56


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.408918,1.660140,0.466667,0.533333,00:13
1,1.216836,1.095374,0.600000,0.400000,00:13
2,1.019377,0.772962,0.716667,0.283333,00:13
3,0.867313,0.578879,0.800000,0.200000,00:13
4,0.726904,0.464777,0.816667,0.183333,00:13
5,0.620236,0.394044,0.866667,0.133333,00:13
6,0.524916,0.349823,0.866667,0.133333,00:13
7,0.448233,0.327718,0.866667,0.133333,00:14
8,0.394627,0.293842,0.883333,0.116667,00:13
9,0.347723,0.257715,0.900000,0.100000,00:13


In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

### Sin Optimizer

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=[accuracy,error_rate], pretrained=True)
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)